In [2]:
pip install music21


  Obtaining dependency information for music21 from https://files.pythonhosted.org/packages/a1/1b/ef7bdf01df19cc7ac9294531a9c991c6d382bde6bc15c9d106b9a5e547ef/music21-9.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpickle from https://files.pythonhosted.org/packages/19/c3/453e4e2da82d5efad9e653916a120d94daf5062f7eae43e28f39fff1bc6a/jsonpickle-3.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for more-itertools from https://files.pythonhosted.org/packages/50/e2/8e10e465ee3987bb7c9ab69efb91d867d93959095f4807db102d07995d94/more_itertools-10.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for webcolors>=1.5 from https://files.pythonhosted.org/packages/d5/e1/3e9013159b4cbb71df9bd7611cbf90dc2c621c8aeeb677fc41dad72f2261/webcolors-1.13-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/22.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/22.8 MB 1.1 MB/s eta 0:00:21
   -------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import numpy as np
from music21 import converter, instrument, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.utils import to_categorical

# Function to process MIDI files and extract notes and chords
def process_midi(file_path):
    notes = []

    midi = converter.parse(file_path)

    notes_to_parse = None

    try:
        # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()
    except:
        # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

# Function to create input sequences and output labels from notes
def prepare_sequences(notes, sequence_length):
    pitch_names = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitch_names))

    network_input = []
    network_output = []

    for i in range(len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape input sequences
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

    # Normalize input
    network_input = network_input / float(len(pitch_names))

    # One-hot encode output labels
    network_output = to_categorical(network_output)

    return network_input, network_output

# Load MIDI files and process them
def load_data(data_dir):
    notes = []

    for file in os.listdir(data_dir):
        if file.endswith('.mid') or file.endswith('.midi'):
            file_path = os.path.join(data_dir, file)
            notes += process_midi(file_path)

    return notes

# Build LSTM model
def create_model(input_shape, num_classes):
    model = Sequential([
        LSTM(512, input_shape=input_shape, return_sequences=True),
        Dropout(0.3),
        LSTM(512, return_sequences=True),
        LSTM(512),
        Dropout(0.3),
        Dense(256),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

# Generate music sequence using the trained model
def generate_music(model, network_input, pitch_names, sequence_length, num_output_notes):
    start = np.random.randint(0, len(network_input) - 1)

    int_to_note = dict((number, note) for number, note in enumerate(pitch_names))

    pattern = network_input[start]
    prediction_output = []

    for _ in range(num_output_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(len(pitch_names))

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

# Parameters
data_dir = 'midi_data'
sequence_length = 100
num_epochs = 100
batch_size = 64

# Load and process MIDI data
notes = load_data(data_dir)
network_input, network_output = prepare_sequences(notes, sequence_length)
pitch_names = sorted(set(item for item in notes))
num_classes = len(pitch_names)

# Create and train LSTM model
model = create_model((network_input.shape[1], network_input.shape[2]), num_classes)
model.fit(network_input, network_output, epochs=num_epochs, batch_size=batch_size)

# Generate music sequence
generated_sequence = generate_music(model, network_input, pitch_names, sequence_length, num_output_notes=100)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'midi_data'